# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź listę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

In [3]:
import psycopg2 as pg
import pandas as pd
connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020');

1. Znajdź listę wszystkich filmów o tej samej długości.

In [4]:
#Usuwam niepotrzebne powtórzenia długosci filmów i sortuje malejąco
df = pd.read_sql("""SELECT DISTINCT length, title
                    FROM film
                    ORDER BY length DESC""",con=connection)
print('Lista wszystkich filmow o tej samej dlugosci:\n', df)

Lista wszystkich filmow o tej samej dlugosci:
      length                title
0       185       Control Anthem
1       185         Worst Banger
2       185    Sweet Brotherhood
3       185       Darn Forrester
4       185          Gangs Pride
..      ...                  ...
995      46         Alien Center
996      46            Iron Moon
997      46     Labyrinth League
998      46  Ridgemont Submarine
999      46        Kwai Homeward

[1000 rows x 2 columns]


C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


2. Znajdź wszystkich klientów mieszkających w tym samym mieście.

In [63]:
#1
#SPRAWDZAM CZY W JAKIMS MIESCIE MIESZKAJA TE SAME OSOBY
#Używajac funkcji COUNT() na podstawie danych w GROUP BY()
df = pd.read_sql("""SELECT  city.city, COUNT(customer.last_name)
                    FROM customer 
                    INNER JOIN address ON customer.address_id = address.address_id 
                    INNER JOIN city ON address.city_id = city.city_id
                    GROUP BY city.city
                    ORDER BY COUNT(customer.last_name) DESC
                    
                """,con=connection)
print('Wszyscy klienci z tych samych miast: \n', df)

#2
#SZUKAM KLIENTOW Z TYCH SAMYCH MIAST
#wprowadzajac wyszukiwanie('WHERE') po zduplikowanych miastach
df = pd.read_sql("""SELECT DISTINCT city.city, customer.first_name, customer.last_name
                    FROM customer 
                    INNER JOIN address ON customer.address_id = address.address_id 
                    INNER JOIN city ON address.city_id = city.city_id
                    WHERE city.city in ('London', 'Aurora')
                    ORDER BY city ASC
                    
                """,con=connection)
print('Wszyscy klienci z tych samych miast: \n\n', df)


Wszyscy klienci z tych samych miast: 
                 city  count
0             London      2
1             Aurora      2
2              Tokat      1
3            Atlixco      1
4           Mukateve      1
..               ...    ...
592       Greensboro      1
593  Ocumare del Tuy      1
594        Southport      1
595         Pemalang      1
596           Taguig      1

[597 rows x 2 columns]
Wszyscy klienci z tych samych miast: 

      city first_name last_name
0  Aurora    Clinton    Buford
1  Aurora      Scott   Shelley
2  London      Cecil     Vines
3  London     Mattie   Hoffman


C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


3. Oblicz średni koszt wypożyczenia wszystkich filmów.

In [24]:
#Uzywajac funkcji AVG() mam możliwość wyliczenia średniego kosztu wszystkich filmów
#Zawierajac ta funkcje na początku (zaraz za SELECT) mam mozliwosc otrzymania odpowiedniego wyniku
df = pd.read_sql("""SELECT AVG(payment.amount) AS payment_amount
                FROM payment 
                INNER JOIN rental ON payment.rental_id = rental.rental_id
                INNER JOIN inventory ON rental.inventory_id = inventory.inventory_id 
                INNER JOIN film ON inventory.film_id = film.film_id
                """,con=connection)
print('Sredni koszt wypozyczenia wszystkich filmow:\n', df)

Sredni koszt wypozyczenia wszystkich filmow:
    payment_amount
0        4.200606


C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.

In [22]:
#Używajac funkcji COUNT() na podstawie danych w GROUP BY() mam zliczenia liczby filmów dla danej kategorii
df = pd.read_sql("""SELECT category.name, COUNT(film.title)
                FROM film 
                INNER JOIN film_category ON film.film_id = film_category.film_id 
                INNER JOIN category ON film_category.category_id = category.category_id
                GROUP BY category.name
                ORDER BY category.name ASC
                """,con=connection)
print("Liczba filmów we wszystkich kategoriach:\n", df)

Liczba filmów we wszystkich kategoriach:
            name  count
0        Action     64
1     Animation     66
2      Children     60
3      Classics     57
4        Comedy     58
5   Documentary     68
6         Drama     62
7        Family     69
8       Foreign     73
9         Games     61
10       Horror     56
11        Music     51
12          New     63
13       Sci-Fi     61
14       Sports     74
15       Travel     57


C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.

In [66]:
#To zadanie wykonuje podobnie jak zadanie powyżej, jednak dla odpowiendich zmiennych
#nie biore pod uwagę kolumny imion, ponieważ mogą się one powtarzać
#bardziej indywidualnym sposobem znalezienia osoby jest kolumna z jego nazwiskiem
df = pd.read_sql("""SELECT country.country AS kraj, COUNT(customer.last_name) AS ilosc_klientow
                    FROM customer 
                    INNER JOIN address ON customer.address_id = address.address_id 
                    INNER JOIN city ON address.city_id = city.city_id 
                    INNER JOIN country ON city.country_id = country.country_id
                    GROUP BY country.country
                    ORDER BY ilosc_klientow DESC
                """,con=connection)
print('Miejsca zamieszkania członkow personelu: \n', df)

Miejsca zamieszkania członkow personelu: 
               kraj  ilosc_klientow
0            India              60
1            China              53
2    United States              36
3            Japan              31
4           Mexico              30
..             ...             ...
103        Moldova               1
104          Tonga               1
105          Nepal               1
106      Greenland               1
107  French Guiana               1

[108 rows x 2 columns]


C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.

In [205]:
#W tym przykładzie zwraca mi pusty data frame
df = pd.read_sql("""SELECT address.address, COUNT(customer.last_name)
                    FROM customer
                    INNER JOIN rental ON customer.customer_id = rental.customer_id
                    INNER JOIN staff ON rental.staff_id = staff.staff_id
                    INNER JOIN store ON staff.store_id = store.store_id
                    INNER JOIN address on store.address_id = address.address_id
                    GROUP BY address.address 
                    HAVING COUNT(customer.last_name) > 100 AND COUNT(customer.last_name) < 300          
                """,con=connection)
print('Informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów: \n', df)

Informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów: 
 Empty DataFrame
Columns: [address, count]
Index: []


C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.

In [21]:
#Robiac odpowiednie polaczenia znajduje odpowienie osoby oraz zliczam sume godzin ogladanych filmow
df = pd.read_sql("""SELECT customer.first_name, customer.last_name, SUM(film.length)
                    FROM rental 
                    INNER JOIN inventory ON rental.inventory_id = inventory.inventory_id 
                    INNER JOIN film ON inventory.film_id = film.film_id
                    INNER JOIN customer ON rental.customer_id = customer.customer_id
                    GROUP BY customer.first_name, customer.last_name
                    ORDER BY SUM(film.length) > 200 
                    """,con=connection)
print('Wszyscy klienci, którzy oglądali filmy ponad 200 godzin: \n', df)

Wszyscy klienci, którzy oglądali filmy ponad 200 godzin: 
     first_name  last_name   sum
0          Sue     Peters  4319
1     Kimberly        Lee  2965
2     Caroline     Bowman  1847
3      Colleen     Burton  2668
4        Hilda    Hopkins  3080
..         ...        ...   ...
594       Leon     Bostic  2864
595      Linda   Williams  2971
596    Johnnie   Chisholm  2835
597   Lawrence     Lawton  3415
598    Michael  Silverman  3145

[599 rows x 3 columns]


8. Oblicz średnią wartość wypożyczenia filmu.

In [20]:
#Wchodzę do katalogu film, gdzie wybieram rental_rate z którego liczę 
#srednia wypozyczenia wszystkich filmow
#składnia AS opisuje nazwe tabeli, bez tego byśmy mieli samo AVG, czytane jako nazwa
df = pd.read_sql("""SELECT AVG(rental_rate) AS rental_rate
                FROM film 
                """,con=connection)
print("Srednia wypozyczenia\n", df)

Srednia wypozyczenia
    rental_rate
0         2.98


C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.

In [27]:
#Łączę odpowiednie katalogi, gdzie liczę później średnia z danej film.lenth, 
# zależnej od category.name na podstawie składni GROUP BY
# Dzięki ORDER BY układam  kategorie alfabetycznie 
df = pd.read_sql("""SELECT category.name,  AVG(film.length) AS film_length
                    FROM film
                    INNER JOIN film_category ON film.film_id = film_category.film_id
                    INNER JOIN category ON film_category.category_id = category.category_id
                    GROUP BY category.name
                    ORDER BY category.name ASC
                """,con=connection)
print("Srednia długość filmu dla danej kategorii: \n", df)

Srednia długość filmu dla danej kategorii: 
            name  film_length
0        Action   111.609375
1     Animation   111.015152
2      Children   109.800000
3      Classics   111.666667
4        Comedy   115.827586
5   Documentary   108.750000
6         Drama   120.838710
7        Family   114.782609
8       Foreign   121.698630
9         Games   127.836066
10       Horror   112.482143
11        Music   113.647059
12          New   111.126984
13       Sci-Fi   108.196721
14       Sports   128.202703
15       Travel   113.315789


C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.

In [5]:
# Szukam najdłużego tytułu filmowego pod względem długości nazwy
# Łączę odpowiednie katalogi i wybieram wymagane dane. 
# Następnie za pomocą ORDER BY sortuje filmy od najdłuższego do najkrótszego.
df = pd.read_sql("""SELECT name, title 
                    FROM film f
                    JOIN film_category f_c ON f.film_id = f_c.film_id 
                    JOIN category c ON f_c.category_id = c.category_id 
                    WHERE char_length(title) = (SELECT MAX(char_length(title)) FROM film 
                    JOIN film_category f_c2 ON film.film_id = f_c2.film_id 
                    JOIN category c2 ON f_c2.category_id = c2.category_id 
                    WHERE c2.name = c.name) 
                    GROUP BY name, title
                """, con=connection)
df.columns = ['category name', 'longest_title']
df

C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,category name,longest_title
0,Action,Entrapment Satisfaction
1,Animation,Telemark Heartbreakers
2,Children,Heartbreakers Bright
3,Children,Microcosmos Paradise
4,Children,Sweethearts Suspects
5,Classics,Extraordinary Conquerer
6,Comedy,Trainspotting Strangers
7,Documentary,Deliverance Mulholland
8,Documentary,Intolerable Intentions
9,Drama,Goldfinger Sensibility


11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

In [28]:
# NAJDŁUŻSZY FILM W KAŻDEJ Z KATEGORII
# To zadanie postanowaiłem rozwiązać w sposób niestety dość nie optymalny, gdyż mysiałem kopiować poprzednio stworzony kod kilkukrotnie.
# Wyszukałem najdłuższego filmu dla danej kategorii dzięki DESC i poprzez LIMIT wyznaczyłem pierwsze z góry najdłuższe  tytuły.
d_Action = pd.read_sql("""SELECT category.name, film.title, film.length
                    FROM film
                    INNER JOIN film_category ON film.film_id = film_category.film_id
                    INNER JOIN category ON film_category.category_id = category.category_id
                    WHERE category.name IN ('Action')
                    ORDER BY film.length DESC LIMIT 2
                """,con=connection)
print("Najdluzsze tytuly filmowe w kategorii 'Action' \n", d_Action)

d_Animation = pd.read_sql("""SELECT category.name, film.title, film.length
                    FROM film
                    INNER JOIN film_category ON film.film_id = film_category.film_id
                    INNER JOIN category ON film_category.category_id = category.category_id
                    WHERE category.name IN ('Animation')
                    ORDER BY film.length DESC limit 2
                """,con=connection)
print("\nNajdluzsze tytuly filmowe w kategorii 'Animation' \n", d_Animation)

d_Children = pd.read_sql("""SELECT category.name, film.title, film.length
                    FROM film
                    INNER JOIN film_category ON film.film_id = film_category.film_id
                    INNER JOIN category ON film_category.category_id = category.category_id
                    WHERE category.name IN ('Children')
                    ORDER BY film.length DESC limit 2 
                """,con=connection)
print("Najdluzsze tytuly filmowe w kategorii 'Children' \n", d_Children)

d_Classics = pd.read_sql("""SELECT category.name, film.title, film.length
                    FROM film
                    INNER JOIN film_category ON film.film_id = film_category.film_id
                    INNER JOIN category ON film_category.category_id = category.category_id
                    WHERE category.name IN ('Classics')
                    ORDER BY film.length DESC limit 1
                """,con=connection)
print("\nNajdluzsze tytuly filmowe w kategorii 'Classics' \n", d_Classics)

d_Comedy = pd.read_sql("""SELECT category.name, film.title, film.length
                    FROM film
                    INNER JOIN film_category ON film.film_id = film_category.film_id
                    INNER JOIN category ON film_category.category_id = category.category_id
                    WHERE category.name IN ('Comedy')
                    ORDER BY film.length DESC limit 1 
                """,con=connection)
print("\nNajdluzsze tytuly filmowe w kategorii 'Comedy' \n", d_Comedy)

d_Documentary = pd.read_sql("""SELECT category.name, film.title, film.length
                    FROM film
                    INNER JOIN film_category ON film.film_id = film_category.film_id
                    INNER JOIN category ON film_category.category_id = category.category_id
                    WHERE category.name IN ('Documentary')
                    ORDER BY film.length DESC limit 2
                """,con=connection)
print("\nNajdluzsze tytuly filmowe w kategorii 'Documentary' \n", d_Documentary)

d_Drama = pd.read_sql("""SELECT category.name, film.title, film.length
                    FROM film
                    INNER JOIN film_category ON film.film_id = film_category.film_id
                    INNER JOIN category ON film_category.category_id = category.category_id
                    WHERE category.name IN ('Drama')
                    ORDER BY film.length DESC limit 2
                """,con=connection)
print("\nNajdluzsze tytuly filmowe w kategorii 'Drama' \n", d_Drama)

d_Family = pd.read_sql("""SELECT category.name, film.title, film.length
                    FROM film
                    INNER JOIN film_category ON film.film_id = film_category.film_id
                    INNER JOIN category ON film_category.category_id = category.category_id
                    WHERE category.name IN ('Family')
                    ORDER BY film.length DESC limit 2
                """,con=connection)
print("\nNajdluzsze tytuly filmowe w kategorii 'Family' \n", d_Family)

d_Foreign = pd.read_sql("""SELECT category.name, film.title, film.length
                    FROM film
                    INNER JOIN film_category ON film.film_id = film_category.film_id
                    INNER JOIN category ON film_category.category_id = category.category_id
                    WHERE category.name IN ('Foreign')
                    ORDER BY film.length DESC limit 2
                """,con=connection)
print("\nNajdluzsze tytuly filmowe w kategorii 'Foreign' \n", d_Foreign)

d_Games = pd.read_sql("""SELECT category.name, film.title, film.length
                    FROM film
                    INNER JOIN film_category ON film.film_id = film_category.film_id
                    INNER JOIN category ON film_category.category_id = category.category_id
                    WHERE category.name IN ('Games')
                    ORDER BY film.length DESC limit 1
                """,con=connection)
print("\nNajdluzsze tytuly filmowe w kategorii 'Games' \n", d_Games)

d_Horror = pd.read_sql("""SELECT category.name, film.title, film.length
                    FROM film
                    INNER JOIN film_category ON film.film_id = film_category.film_id
                    INNER JOIN category ON film_category.category_id = category.category_id
                    WHERE category.name IN ('Horror')
                    ORDER BY film.length DESC limit 2
                """,con=connection)
print("\nNajdluzsze tytuly filmowe w kategorii 'Horror' \n", d_Horror)

d_Music = pd.read_sql("""SELECT category.name, film.title, film.length
                    FROM film
                    INNER JOIN film_category ON film.film_id = film_category.film_id
                    INNER JOIN category ON film_category.category_id = category.category_id
                    WHERE category.name IN ('Music')
                    ORDER BY film.length DESC limit 1
                """,con=connection)
print("\nNajdluzsze tytuly filmowe w kategorii 'Music' \n", d_Music)

d_New = pd.read_sql("""SELECT category.name, film.title, film.length
                    FROM film
                    INNER JOIN film_category ON film.film_id = film_category.film_id
                    INNER JOIN category ON film_category.category_id = category.category_id
                    WHERE category.name IN ('New')
                    ORDER BY film.length DESC limit 1
                """,con=connection)
print("\nNajdluzsze tytuly filmowe w kategorii 'New' \n", d_New)

d_Sci_Fi = pd.read_sql("""SELECT category.name, film.title, film.length
                    FROM film
                    INNER JOIN film_category ON film.film_id = film_category.film_id
                    INNER JOIN category ON film_category.category_id = category.category_id
                    WHERE category.name IN ('Sci-Fi')
                    ORDER BY film.length DESC limit 1
                """,con=connection)
print("\nNajdluzsze tytuly filmowe w kategorii 'Sci-Fi' \n", d_Sci_Fi)

d_Sports = pd.read_sql("""SELECT category.name, film.title, film.length
                    FROM film
                    INNER JOIN film_category ON film.film_id = film_category.film_id
                    INNER JOIN category ON film_category.category_id = category.category_id
                    WHERE category.name IN ('Sports')
                    ORDER BY film.length DESC limit 1
                """,con=connection)
print("\nNajdluzsze tytuly filmowe w kategorii 'Sports' \n", d_Sports)

d_Travel = pd.read_sql("""SELECT category.name, film.title, film.length
                    FROM film
                    INNER JOIN film_category ON film.film_id = film_category.film_id
                    INNER JOIN category ON film_category.category_id = category.category_id
                    WHERE category.name IN ('Travel')
                    ORDER BY film.length DESC limit 2
                """,con=connection)
print("\nNajdluzsze tytuly filmowe w kategorii 'Travel' \n", d_Travel)

#Poniżej znajdują się wyniki moich obliczeń dla danej kategorii


kat = ["Sport"]

C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or

Najdluzsze tytuly filmowe w kategorii 'Action' 
      name           title  length
0  Action  Darn Forrester     185
1  Action    Worst Banger     185

Najdluzsze tytuly filmowe w kategorii 'Animation' 
         name         title  length
0  Animation   Gangs Pride     185
1  Animation  Pond Seattle     185
Najdluzsze tytuly filmowe w kategorii 'Children' 
        name           title  length
0  Children     Fury Murder     178
1  Children  Wrong Behavior     178

Najdluzsze tytuly filmowe w kategorii 'Classics' 
        name              title  length
0  Classics  Conspiracy Spirit     184

Najdluzsze tytuly filmowe w kategorii 'Comedy' 
      name           title  length
0  Comedy  Control Anthem     185

Najdluzsze tytuly filmowe w kategorii 'Documentary' 
           name           title  length
0  Documentary       Wife Turn     183
1  Documentary  Young Language     183

Najdluzsze tytuly filmowe w kategorii 'Drama' 
     name           title  length
0  Drama   Jacket Frisco     1

C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or

     name               title  length
0  Sci-Fi  Soldiers Evolution     185

Najdluzsze tytuly filmowe w kategorii 'Sports' 
      name            title  length
0  Sports  Smoochy Control     184

Najdluzsze tytuly filmowe w kategorii 'Travel' 
      name              title  length
0  Travel      Muscle Bright     185
1  Travel  Sweet Brotherhood     185


C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\marci\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [7]:
import main